In [1]:
import tensorflow as tf
import keras
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Rescaling
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from skimage import color
from matplotlib.pyplot import imshow
from scipy import spatial

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

# we get local data_path
data_path = ''

classes_index = 0
label_list = []
        
class Dataset(object):
    def __init__(self, data_path):
        self.data_path = data_path
        assert os.path.exists(self.data_path), 'Insert a valid path!'

        # get class list
        self.data_classes = os.listdir(self.data_path)

        # init mapping dict
        self.data_mapping = {}

        # populate mapping dict
        for c, c_name in enumerate(self.data_classes):
            temp_path = os.path.join(self.data_path, c_name)
            temp_images = os.listdir(temp_path)

            for i in temp_images:
                img_tmp = os.path.join(temp_path, i)

                if img_tmp.lower().endswith(('.jpg', '.jpeg')):
                    if c_name == 'distractor':
                        self.data_mapping[img_tmp] = -1
                    else:
                        self.data_mapping[img_tmp] = c_name

        print('Loaded {:d} from {:s} images'.format(len(self.data_mapping.keys()),
                                                    self.data_path))
        
    def get_data_paths(self):
        # returns a list of imgpaths and related classes
        images = []
        classes = []
        for img_path in self.data_mapping.keys():
            if img_path.lower().endswith(('.jpg', '.jpeg')):
                images.append(img_path)
                classes.append(self.data_mapping[img_path])
        return images, np.array(classes)

    def get_data_images(self):
        t_size = (224,224)
        # returns a list of imgpaths and related classes      
        images = []
        for img_path in self.data_mapping.keys():
            if img_path.lower().endswith(('.jpg', '.jpeg')):
                img = image.load_img(img_path,target_size=t_size) 
                img = img_to_array(img)
                img = np.asarray(img)
                img = color.rgb2gray(img)
                #print('arrayImg', img)
                images.append(img)
        return np.array(images)

    def get_data_classes(self, classes_index, label_list):
        # returns a list of imgpaths and related classes
        classes = []
        for img_path in self.data_mapping.keys():
            if img_path.lower().endswith(('.jpg', '.jpeg')):
                if self.data_mapping[img_path] in label_list:
                    classes.append(label_list.index(self.data_mapping[img_path]))
                else:
                    label_list.append(self.data_mapping[img_path])
                    classes.append(classes_index)
                    classes_index += 1
        print(label_list)
        return np.array(classes), classes_index

    def num_classes(self):
        # returns number of classes of the dataset
        return len(self.data_classes)

# we define training_path
training_path = os.path.join(data_path, 'training')

# we define validation path, query and gallery
validation_path = os.path.join(data_path, 'validation')
gallery_path = os.path.join(validation_path, 'gallery')
query_path = os.path.join(validation_path, 'query')


training_dataset = Dataset(data_path=training_path)
gallery_dataset = Dataset(data_path=gallery_path)
query_dataset = Dataset(data_path=query_path)

training_paths, training_classes = training_dataset.get_data_paths()

# we get validation gallery and query data

gallery_paths, gallery_classes = gallery_dataset.get_data_paths()
query_paths, query_classes = query_dataset.get_data_paths()



In [ ]:
X = training_dataset.get_data_images()
y, classes_index = training_dataset.get_data_classes(classes_index, label_list)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=0)

train_images =Xtrain
train_labels = ytrain
test_images = Xtest
test_labels = ytest

# Normalize the images.
#train_images = (train_images / 255)
#test_images = (test_images / 255) - 0.5

# Reshape the images.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(train_images[i])
    plt.title(int(train_labels[i]))
    plt.axis("off")

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(64, 5, activation='relu', padding='same'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(2))

model.add(keras.layers.Conv2D(64, 5, activation='relu', padding='same'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(2))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(480, activation='relu'))

model.add(keras.layers.Dense(10, activation='softmax'))

opt= Adam() # iniziale: 0.001

# Compile the model.
model.compile(
  optimizer=opt,
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
epochs = 20
with tf.device("/gpu:0"):
  history = model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=epochs,
    validation_data=(test_images, to_categorical(test_labels)),
    callbacks=[keras.callbacks.EarlyStopping(patience=8, verbose=1, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(factor=.5, patience=4, verbose=1)] )
  
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

# Predict on the first 5 test images
predictions = model.predict(test_images[:5])

# Print our model's predictions
print(np.argmax(predictions, axis=1))

# Check our predictions
print(test_labels[:5])



In [ ]:
model.save("")

In [66]:
#model = keras.models.load_model('/Users/damianoduranti/Desktop/v2.0')
#model.summary()

In [ ]:
model.layers.pop()

In [ ]:
model.summary()

In [ ]:
gallery_img = gallery_dataset.get_data_images()
gallery_labels, classes_index = gallery_dataset.get_data_classes(classes_index=classes_index, label_list=label_list)

In [ ]:
query_img = query_dataset.get_data_images()
query_labels, classes_index = query_dataset.get_data_classes(classes_index=classes_index, label_list=label_list)

In [107]:
# Predict on the first 5 test images.
query_prediction = model.predict(query_img)


In [ ]:
print(len(query_prediction))
np.set_printoptions(suppress=True)

In [ ]:
query_idx = 0
distances = []
np.set_printoptions(suppress=True)
for i in range (len(query_prediction)):
    d = 1-spatial.distance.cosine(query_prediction[query_idx], predictions[i])
    distances.append([d, i])

#print(distances)
    
distances.sort(reverse=True)    
print("True class: "+str(query_labels[query_idx]))
print("Predicted class: "+str(gallery_labels[distances[0][1]]))

plt.imshow(query_set[80][0].squeeze())
plt.title(os.path.dirname(generator1.filenames[0]))

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(10):
    ax = plt.subplot(2,5, i + 1)
    plt.imshow(gallery_img[distances[i][1]])
    plt.axis("off")

In [ ]:
top1 = 0
top5 = 0
top10 = 0
non_top = 0
for j in range (len(query_img)):   
    distances = []
    np.set_printoptions(suppress=True)
    for i in range (len(predictions)):
        d = 1-spatial.distance.cosine(query_prediction[j], predictions[i])
        distances.append([d, i])
        
    distances.sort(reverse=True)

    q_label = int(query_labels[j])
    top_ten = distances[:10]
    isthere = False
    for k in range (len(top_ten)):
        if int(gallery_labels[top_ten[k][1]]) == q_label:
            isthere = True
            if (k+1) == 1:
                top1+=1
                break
            elif (k+1) <=5:
                top5+=1
                break
            elif (k+1) <= 10:
                top10+=1
                break
    if not isthere:
        non_top +=1
        plt.imshow(query_img[j])
        plt.title(int(query_labels[j]))
        print(f"Query number {j} has no top-10 appearences")
        
            
print("top1: "+ str(top1))
print("top1: "+ str(top5))
print("top1: "+ str(top10))
print("Non top: " + str(non_top))
top1 = top1/len(query_img)*100
top5 = top5/len(query_img)*100 + top1
top10 = top10/len(query_img)*100 + top5
print(len(query_img))
print("top1: " + str(top1) + "%")
print("top5: " + str(top5)+ "%")
print("top10: " + str(top10)+ "%")
